In [1]:
import os
import shutil
import numpy
import pandas
import c3d
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

In [11]:
#salviamo i file con indici crescenti da 0
#questo ha i seguenti vantaggi:
#possiamo generare con un xrange tutti i nomi di file
#possiamo prendere una permutazione semplicemente permutando il range
#usiamo l ordine naturale per la corrispondenza con la classe(file 3.c3d->indice 4 delle labels)

#PERCORSI RELATIVI!

#cartella base
#qui mi aspetto dentro le dat_folders
BASE_FOLDER = "data/"
#cartella con gli output
BASE_FOLDER_OUT = "processed/"
#classi 0-3(faremo +1 al massimo)
#cartelle 0-3
#dentro le data folders ho i rispettivi contenuti degli zip(cartelle con nome cognome)
DATA_FOLDERS = ["0/","1/","2/","3/"]
#classi associate alle data_folders
CLASSES = [i for i in range(len(DATA_FOLDERS))]
#parametri di interesse(presi da ferrari)
PARAMS = ["C7","REP","RUL","RASIS","RPSIS","RCA","RGT","RLE","RCA",
         "RFM","RA","LEP","LUL","LASIS","LPSIS","LCA","LGT","LLE",
         "LFM","LA"]
NUM_PARAMS = len(PARAMS)

#frame vicon
FRAME_VICOM = 100
#frame che vogliamo ogni secondo
FRAME_PER_SECOND = 25
#guardiamo che sia divisibile
assert FRAME_VICOM % FRAME_PER_SECOND == 0
#campiono ogni FRAME_VICOM/FRAME_PER_SECOND
FRAME_COUNT = FRAME_VICOM/FRAME_PER_SECOND
#NUMERO SEQUENZE
SEQ_NUM = 20
#LUNGHEZZA SEQUENZE
SEQ_LEN = 10
#se usare MinMax
minMax = True

#lista classi che salveremo in un file a parte
#lista perche non ho voglia di contare tutti i file,e piu lenta ma tanto lo lanciamo solo una volta
j = 0
len_step = []
for i in range(len(CLASSES)):
    #entro nella cartella classe
    folder_class = BASE_FOLDER+DATA_FOLDERS[i]
    print "start working in folder {}".format(folder_class)
    #per ogni cartella paziente estraggo i c3d
    for folder_patient in os.listdir(folder_class):
        folder_patient = folder_class+folder_patient
        #lista file
        for file_name in [file_name for file_name  in os.listdir(folder_patient) if file_name[-3:] == "c3d"]:
            #questo file ha un errore nell header!
            if file_name == "CS_preOp_sc_gait05.c3d":
                continue
            #reader per leggere il c3d
            file_name = folder_patient+"/"+file_name
            reader = c3d.Reader(open(file_name,'rb'))
            first_frame = reader.header.first_frame
            last_frame = reader.header.last_frame
            
            context_header = [s.strip() for s in reader.groups['EVENT'].params['CONTEXTS'].string_array]
            label_header =[s.strip() for s in reader.groups['EVENT'].params['LABELS'].string_array]
            times_header = reader.groups['EVENT'].params['TIMES'].float_array
            times_header = times_header.flatten()
            times_header = times_header[times_header!=0]
            df = pandas.DataFrame(data={"context": context_header,"label":label_header, "times":times_header})
            df = df.sort_values(by="times")
            df.times = (df.times*100).astype("int32")
            #scartiamo se non ci sono abbastanza eventi interessanti
            if len(df.times) < 9 :
                continue
            #prendiamo dal occ
            occ = 0
            frame = df[(df.context== "Right") & (df.label== "Foot Strike")].iloc[occ].times
            #scartiamo se il frame è negativo
            # TODO controlla gabri
            if frame-first_frame <0:
                continue

            

            #for group in reader.groups:
            #    print group
            #    print reader.get(group).params
            #    print "---"
            
            #estraggo labels dei points
            params = [param.strip() for param in reader.get("POINT").get("LABELS").string_array]
            #guardo che ci siano tutti quelli del file matlab di Ferrari
            #sono 52 quelli che non l hanno
            #visto che ogni paziente ha piu prove non dovrebbe essere un problema
            if len([param for param in PARAMS if param not in params]) > 0:
                continue
            #ci sono tutti ma non so in che ordine!
            #mi servono gli indici dei param in params
            indexes = numpy.array([params.index(param) for param in PARAMS])
            #li usero per accedere ai frame
            #ogni frame ha NUM_PARAMS*5 coordinate(xyz + precisione e altro)
            #devo salvare le sequenze
            #salvo un elemento in piu che usero come label
            #ho numero_sequenze*lunghezza_sequenza+1*numero_marker*3

            index_camp = FRAME_COUNT
            index =0
            #questo serve per memoria del precendente
            point_numpy = None
            #se ho SEQ_NUM da SEQ_LEN mi servono in tutto SEQ_LEN + SEQ_NUMBER
            #ho anche la label dell'ultima cosi
            points_numpy = numpy.zeros((SEQ_LEN+SEQ_NUM,NUM_PARAMS,3))

            for k,points,analog in reader.read_frames():
                if k >= frame:
                    #prendo un campione solo se index_camp = FRAME_COUNT
                    if index_camp == FRAME_COUNT:
                        #campiono
                        #prendo la differenza con il precedente
                        if point_numpy is None:
                            points_numpy[index,:,:] = 0
                            #seq_numpy[index_seq_num,index_seq_len,:,0:3] = 0.0
                            point_numpy = points[indexes,0:3]
                        else:
                            points_numpy[index,:,:] = points[indexes,0:3] - point_numpy
                            #seq_numpy[index_seq_num,index_seq_len,:,0:3] = points[indexes,0:3] - point_numpy
                            point_numpy = points[indexes,0:3]

                        index +=1
                        index_camp = 0
                        if index == points_numpy.shape[0]:
                            break
                    else:
                        index_camp +=1
            #potrei essere qui ma non avere tutti i dati
            if index != points_numpy.shape[0]:
                print "non ci sono abbastanza dati in {}! indice {} su {},k {} su {}".format(file_name,index,points_numpy.shape[0],
                                                                                      frame,k)
            else:
                #creo le sequenze
                seq_numpy = numpy.zeros((SEQ_NUM,SEQ_LEN+1,NUM_PARAMS,3))
                for i in xrange(SEQ_NUM):
                    seq_numpy[i,:,:,:] = points_numpy[i:i+(SEQ_LEN+1),:,:]
                #uso minMax tra -1 e 1
                if minMax:
                    seq_numpy = MinMaxScaler(feature_range=(-1,1)).fit_traform(seq_numpy.reshape(-1,NUM_PARAMS*3)).reshape(seq_numpy.shape)
                #salvo dati e labels
                data = seq_numpy[:,:-1,:,:]
                labels = seq_numpy[:,-1:,:,:]
                numpy.savez(BASE_FOLDER_OUT+"{}.c3d".format(j),data,labels)
                j+=1

    print "end working in folder {}".format(folder_class)
    print "-------"


print "end"

start working in folder data/0/
end working in folder data/0/
-------
start working in folder data/1/
end working in folder data/1/
-------
start working in folder data/2/
end working in folder data/2/
-------
start working in folder data/3/
end working in folder data/3/
-------
end
